# NetworkKit Accessibility Test

Testing NetworkKit integration for park accessibility analysis to see if we can get significant performance improvements.

In [ ]:
# Import required libraries
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from shapely.geometry import Point
from scipy.spatial import ConvexHull

# Try to import NetworkKit
try:
    import networkit as nk
    print(f"✅ NetworkKit version {nk.__version__} available")
    NETWORKIT_AVAILABLE = True
except ImportError:
    print("❌ NetworkKit not available")
    NETWORKIT_AVAILABLE = False

## 1. Load Test Data (Rotterdam)

Use the same test location as the main app.

In [ ]:
# Rotterdam coordinates
lat, lng = 51.9225, 4.47917
distance = 2000  # 2km radius

print(f"Loading street network for Rotterdam ({lat}, {lng}) within {distance}m...")

# Get parks and green spaces
tags = {"leisure": ["park", "recreation_ground"], "landuse": ["grass", "recreation_ground"]}
parks = ox.features_from_point((lat, lng), tags=tags, dist=distance)

print(f"Found {len(parks)} parks/green spaces")

# Get street network focused around parks (smaller area for testing)
if len(parks) > 0:
    # Calculate bounding box of parks
    bounds = parks.total_bounds  # [minx, miny, maxx, maxy]
    
    # Add buffer around parks
    buffer = 0.01  # ~1km buffer
    north = bounds[3] + buffer
    south = bounds[1] - buffer
    east = bounds[2] + buffer
    west = bounds[0] - buffer
    
    bbox = (west, south, east, north)
    graph = ox.graph_from_bbox(bbox, network_type='walk')
else:
    # Fallback to point-based
    graph = ox.graph_from_point((lat, lng), network_type='walk', dist=1000)

print(f"Street network: {len(graph.nodes())} nodes, {len(graph.edges())} edges")

# Convert to undirected
graph_undirected = ox.convert.to_undirected(graph)
nodes, edges = ox.graph_to_gdfs(graph)

print(f"Undirected network: {len(graph_undirected.nodes())} nodes, {len(graph_undirected.edges())} edges")

## 2. Set Up Analysis Parameters

In [ ]:
# 15-minute walking distance
walking_speed_ms = (4.5 * 1000) / 60  # 4.5 km/h in meters per minute
max_distance = 15 * walking_speed_ms  # ~1125 meters

print(f"🎯 Distance threshold: {max_distance:.1f} meters ({max_distance/1000:.2f} km)")

# Get all nodes
all_nodes = list(graph_undirected.nodes())
print(f"Total nodes for analysis: {len(all_nodes)}")

# For testing, let's use a smaller sample
sample_size = min(50, len(all_nodes))  # Test with 50 nodes first
test_nodes = all_nodes[:sample_size]
print(f"Testing with {len(test_nodes)} sample nodes")

## 3. NetworkX Baseline Implementation

In [ ]:
def calculate_accessibility_networkx(graph, test_nodes, max_distance, nodes_gdf):
    """Calculate accessibility using NetworkX ego graphs"""
    
    start_time = time.time()
    accessibility = {}
    
    print(f"\n🔄 NetworkX calculation for {len(test_nodes)} nodes...")
    
    for i, node in enumerate(test_nodes):
        try:
            # Get ego graph within walking distance
            ego_graph = nx.ego_graph(graph, node, radius=max_distance, distance='length')
            reachable_nodes = [n for n in ego_graph.nodes() if n != node]
            
            # Calculate area (simplified for testing)
            if len(reachable_nodes) >= 3:
                area = calculate_area_from_nodes(reachable_nodes, nodes_gdf)
                accessibility[node] = area
            else:
                accessibility[node] = 0
                
        except Exception as e:
            accessibility[node] = 0
            
        # Progress update
        if (i + 1) % 10 == 0:
            print(f"  Progress: {i+1}/{len(test_nodes)}")
    
    elapsed = time.time() - start_time
    print(f"✅ NetworkX completed in {elapsed:.2f} seconds")
    
    return accessibility, elapsed

def calculate_area_from_nodes(reachable_nodes, nodes_gdf):
    """Calculate area using convex hull"""
    try:
        if len(reachable_nodes) < 3:
            return 0
        
        # Get coordinates
        points = []
        for node in reachable_nodes:
            if node in nodes_gdf.index:
                point = nodes_gdf.loc[node].geometry
                points.append([point.x, point.y])
        
        if len(points) < 3:
            return 0
        
        # Convex hull
        points_array = np.array(points)
        hull = ConvexHull(points_array)
        area_deg_sq = hull.volume  # 2D area
        area_sq_km = area_deg_sq * (111 ** 2)
        area_sq_m = area_sq_km * 1e6
        
        return area_sq_m
        
    except Exception:
        return 0

# Run NetworkX baseline
nx_results, nx_time = calculate_accessibility_networkx(graph_undirected, test_nodes, max_distance, nodes)

## 4. NetworkKit Implementation

In [ ]:
def convert_networkx_to_networkit(nx_graph):
    """Convert NetworkX graph to NetworkKit with proper node mapping"""
    
    print("Converting NetworkX to NetworkKit...")
    
    # Create node mapping (NetworkKit uses integer IDs starting from 0)
    nx_nodes = list(nx_graph.nodes())
    node_to_nk = {node: i for i, node in enumerate(nx_nodes)}
    nk_to_node = {i: node for node, i in node_to_nk.items()}
    
    # Create NetworkKit graph
    nk_graph = nk.Graph(n=len(nx_nodes), weighted=True, directed=False)
    
    # Add edges with weights
    edges_added = 0
    for u, v, data in nx_graph.edges(data=True):
        weight = data.get('length', 100.0)
        nk_graph.addEdge(node_to_nk[u], node_to_nk[v], weight)
        edges_added += 1
    
    print(f"✅ NetworkKit graph created: {nk_graph.numberOfNodes()} nodes, {nk_graph.numberOfEdges()} edges")
    print(f"   Added {edges_added} edges from NetworkX")
    
    return nk_graph, node_to_nk, nk_to_node

def test_networkit_connectivity(nk_graph, test_node_nk):
    """Test basic NetworkKit connectivity"""
    
    print(f"\n🧪 Testing NetworkKit connectivity from node {test_node_nk}...")
    
    try:
        # Test Dijkstra
        dijkstra = nk.distance.Dijkstra(nk_graph, test_node_nk)
        dijkstra.run()
        
        # Test a few distances
        test_targets = list(range(min(10, nk_graph.numberOfNodes())))
        distances_found = 0
        close_nodes = 0
        
        for target in test_targets:
            if target != test_node_nk:
                try:
                    dist = dijkstra.distance(target)
                    if dist < float('inf'):
                        distances_found += 1
                        if dist <= 500:  # Within 500m
                            close_nodes += 1
                except Exception as e:
                    print(f"    Error getting distance to {target}: {e}")
        
        print(f"   ✅ Could calculate distances to: {distances_found}/{len(test_targets)-1} test nodes")
        print(f"   ✅ Nodes within 500m: {close_nodes}")
        
        return distances_found > 0
        
    except Exception as e:
        print(f"   ❌ NetworkKit connectivity test failed: {e}")
        return False

def calculate_accessibility_networkit(nk_graph, node_to_nk, nk_to_node, test_nodes, max_distance, nodes_gdf):
    """Calculate accessibility using NetworkKit"""
    
    start_time = time.time()
    accessibility = {}
    
    print(f"\n🚀 NetworkKit calculation for {len(test_nodes)} nodes...")
    
    for i, node in enumerate(test_nodes):
        try:
            nk_node = node_to_nk[node]
            
            # Use Dijkstra for single-source shortest path
            dijkstra = nk.distance.Dijkstra(nk_graph, nk_node)
            dijkstra.run()
            
            # Find reachable nodes within distance
            reachable_nodes = []
            
            for target_nk in range(nk_graph.numberOfNodes()):
                if target_nk != nk_node:
                    try:
                        distance = dijkstra.distance(target_nk)
                        if distance <= max_distance and distance < float('inf'):
                            target_node = nk_to_node[target_nk]
                            reachable_nodes.append(target_node)
                    except:
                        continue
            
            # Calculate area
            if len(reachable_nodes) >= 3:
                area = calculate_area_from_nodes(reachable_nodes, nodes_gdf)
                accessibility[node] = area
            else:
                accessibility[node] = 0
                
        except Exception as e:
            accessibility[node] = 0
            if i < 3:  # Show errors for first few nodes
                print(f"    Error for node {node}: {e}")
        
        # Progress update
        if (i + 1) % 10 == 0:
            print(f"  Progress: {i+1}/{len(test_nodes)}")
    
    elapsed = time.time() - start_time
    print(f"✅ NetworkKit completed in {elapsed:.2f} seconds")
    
    return accessibility, elapsed

# Run NetworkKit implementation if available
if NETWORKIT_AVAILABLE:
    # Convert graph
    nk_graph, node_to_nk, nk_to_node = convert_networkx_to_networkit(graph_undirected)
    
    # Test connectivity
    if test_nodes:
        test_node = test_nodes[0]
        test_node_nk = node_to_nk[test_node]
        connectivity_ok = test_networkit_connectivity(nk_graph, test_node_nk)
        
        if connectivity_ok:
            # Run full calculation
            nk_results, nk_time = calculate_accessibility_networkit(
                nk_graph, node_to_nk, nk_to_node, test_nodes, max_distance, nodes
            )
        else:
            print("❌ NetworkKit connectivity test failed - skipping full calculation")
            nk_results, nk_time = {}, float('inf')
    else:
        print("❌ No test nodes available")
        nk_results, nk_time = {}, float('inf')
else:
    print("❌ NetworkKit not available")
    nk_results, nk_time = {}, float('inf')

## 5. Compare Results

In [ ]:
print("\n" + "="*60)
print("PERFORMANCE COMPARISON")
print("="*60)

print(f"Sample size: {len(test_nodes)} nodes")
print(f"Distance threshold: {max_distance:.1f}m")
print()

print(f"NetworkX time:  {nx_time:.2f} seconds")
if NETWORKIT_AVAILABLE and nk_time < float('inf'):
    print(f"NetworkKit time: {nk_time:.2f} seconds")
    speedup = nx_time / nk_time if nk_time > 0 else float('inf')
    print(f"Speedup: {speedup:.1f}x")
else:
    print(f"NetworkKit: Not available or failed")

print()
print("Results summary:")
nx_nonzero = sum(1 for v in nx_results.values() if v > 0)
nx_avg = np.mean(list(nx_results.values())) if nx_results else 0

print(f"NetworkX - Non-zero results: {nx_nonzero}/{len(nx_results)}")
print(f"NetworkX - Average area: {nx_avg/1e6:.4f} km²")

if NETWORKIT_AVAILABLE and nk_results:
    nk_nonzero = sum(1 for v in nk_results.values() if v > 0)
    nk_avg = np.mean(list(nk_results.values())) if nk_results else 0
    
    print(f"NetworkKit - Non-zero results: {nk_nonzero}/{len(nk_results)}")
    print(f"NetworkKit - Average area: {nk_avg/1e6:.4f} km²")
    
    # Check if results are similar
    common_nodes = set(nx_results.keys()) & set(nk_results.keys())
    if common_nodes:
        differences = []
        for node in common_nodes:
            diff = abs(nx_results[node] - nk_results[node])
            differences.append(diff)
        
        avg_diff = np.mean(differences)
        max_diff = max(differences)
        
        print(f"\nResult comparison:")
        print(f"Average difference: {avg_diff/1e6:.6f} km²")
        print(f"Maximum difference: {max_diff/1e6:.6f} km²")
        
        if avg_diff < 1000:  # Less than 1000 m² difference
            print("✅ Results are very similar!")
        elif avg_diff < 10000:  # Less than 0.01 km²
            print("✅ Results are reasonably similar")
        else:
            print("⚠️ Results differ significantly")

## 6. Detailed Analysis of First Few Results

In [ ]:
print("\n" + "="*60)
print("DETAILED RESULTS (First 10 nodes)")
print("="*60)

results_df = pd.DataFrame({
    'Node': test_nodes[:10],
    'NetworkX_m2': [nx_results.get(node, 0) for node in test_nodes[:10]],
    'NetworkX_km2': [nx_results.get(node, 0)/1e6 for node in test_nodes[:10]]
})

if NETWORKIT_AVAILABLE and nk_results:
    results_df['NetworkKit_m2'] = [nk_results.get(node, 0) for node in test_nodes[:10]]
    results_df['NetworkKit_km2'] = [nk_results.get(node, 0)/1e6 for node in test_nodes[:10]]
    results_df['Difference_m2'] = results_df['NetworkKit_m2'] - results_df['NetworkX_m2']
    results_df['Difference_km2'] = results_df['Difference_m2'] / 1e6

print(results_df.to_string(index=False, float_format='%.6f'))

## 7. Scaling Test (if NetworkKit works well)

In [ ]:
if NETWORKIT_AVAILABLE and nk_results and nx_time > 0 and nk_time > 0:
    print("\n" + "="*60)
    print("SCALING PROJECTION")
    print("="*60)
    
    sample_size = len(test_nodes)
    total_nodes = len(all_nodes)
    
    # Project times for full network
    nx_projected = (nx_time / sample_size) * total_nodes
    nk_projected = (nk_time / sample_size) * total_nodes
    
    print(f"Current sample: {sample_size} nodes")
    print(f"Full network: {total_nodes} nodes")
    print(f"Scale factor: {total_nodes/sample_size:.1f}x")
    print()
    print(f"Projected NetworkX time: {nx_projected/60:.1f} minutes")
    print(f"Projected NetworkKit time: {nk_projected/60:.1f} minutes")
    print(f"Time savings: {(nx_projected-nk_projected)/60:.1f} minutes")
    
    if nx_projected > 600:  # More than 10 minutes
        print("\n🚀 NetworkKit would provide significant benefit for large networks!")
    else:
        print("\n💡 Both methods are fast enough for this network size")
else:
    print("\n❌ Cannot project scaling - NetworkKit test incomplete")

## 8. Conclusions

In [ ]:
print("\n" + "="*60)
print("CONCLUSIONS")
print("="*60)

if not NETWORKIT_AVAILABLE:
    print("❌ NetworkKit is not installed")
    print("   Install with: pip install networkit")
elif nk_time == float('inf'):
    print("❌ NetworkKit failed to run properly")
    print("   Check NetworkKit installation and graph conversion")
elif nk_time > nx_time:
    print("⚠️ NetworkKit was slower than NetworkX for this small test")
    print("   NetworkKit benefits appear at larger network sizes")
elif nx_nonzero == 0:
    print("⚠️ No accessibility results found - check distance threshold")
    print(f"   Current threshold: {max_distance:.0f}m might be too small")
elif nk_nonzero == 0:
    print("⚠️ NetworkKit produced no results - check implementation")
else:
    speedup = nx_time / nk_time if nk_time > 0 else 0
    print(f"✅ NetworkKit successfully calculated accessibility")
    print(f"✅ Speedup: {speedup:.1f}x for {len(test_nodes)} nodes")
    print(f"✅ Both methods found {nx_nonzero} and {nk_nonzero} non-zero results")
    
    if speedup > 2:
        print("\n🚀 RECOMMENDATION: Integrate NetworkKit into main app")
        print("   Expected significant performance improvement for large networks")
    else:
        print("\n💡 RECOMMENDATION: NetworkX is sufficient for current use")
        print("   Consider NetworkKit only for very large networks (>5000 nodes)")

print("\n" + "="*60)